In [1]:
from pyspark.sql import SparkSession

In [2]:
MAX_MEMORY = "5g"
spark = SparkSession.builder.master("local").appName("movie_recommendation")\
    .config("spark.executor.memory", MAX_MEMORY)\
    .config("spark.driver.memory", MAX_MEMORY)\
    .getOrCreate()

In [3]:
ratings_file = "/Users/ji/data-engineering/01-spark/data/ml-25m/ratings.csv"
ratings_df = spark.read.csv(f"file:///{ratings_file}", inferSchema=True, header=True)

In [4]:
ratings_df.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



In [5]:
ratings_df = ratings_df.select(["userID", "movieID", "rating"])

In [6]:
ratings_df.printSchema()

root
 |-- userID: integer (nullable = true)
 |-- movieID: integer (nullable = true)
 |-- rating: double (nullable = true)



In [7]:
ratings_df.select("rating").describe().show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|          25000095|
|   mean| 3.533854451353085|
| stddev|1.0607439611423508|
|    min|               0.5|
|    max|               5.0|
+-------+------------------+



In [8]:
train_df, test_df = ratings_df.randomSplit([0.8, 0.2])

In [9]:
from pyspark.ml.recommendation import ALS

In [10]:
als = ALS(maxIter=3,
          regParam=0.1,
          userCol="userID",
          itemCol="movieID",
          ratingCol='rating',
          coldStartStrategy="drop")

In [11]:
model = als.fit(train_df)

22/06/13 01:39:57 WARN BlockManager: Block rdd_39_6 could not be removed as it was not found on disk or in memory
22/06/13 01:39:57 WARN BlockManager: Block rdd_39_1 could not be removed as it was not found on disk or in memory
22/06/13 01:39:57 WARN BlockManager: Block rdd_39_0 could not be removed as it was not found on disk or in memory
22/06/13 01:39:57 WARN BlockManager: Block rdd_40_1 could not be removed as it was not found on disk or in memory
22/06/13 01:39:57 WARN BlockManager: Block rdd_40_6 could not be removed as it was not found on disk or in memory
22/06/13 01:39:57 WARN BlockManager: Block rdd_40_0 could not be removed as it was not found on disk or in memory
22/06/13 01:39:57 ERROR Executor: Exception in task 1.0 in stage 9.0 (TID 37)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3865)
	at scala.collection.mutable.ArrayBuilder$ofFloat.mkArray(ArrayBuilder.scala:471)
	at scala.collection.mutable.ArrayBuilder$ofFloat.result

22/06/13 01:39:58 WARN TaskSetManager: Lost task 6.0 in stage 9.0 (TID 42) (localhost executor driver): java.lang.OutOfMemoryError: Java heap space

22/06/13 01:39:58 ERROR TaskSetManager: Task 6 in stage 9.0 failed 1 times; aborting job
22/06/13 01:39:58 ERROR Inbox: Ignoring error
java.util.concurrent.RejectedExecutionException: Task org.apache.spark.executor.Executor$TaskRunner@2569459c rejected from java.util.concurrent.ThreadPoolExecutor@736b87aa[Shutting down, pool size = 7, active threads = 7, queued tasks = 0, completed tasks = 37]
	at java.base/java.util.concurrent.ThreadPoolExecutor$AbortPolicy.rejectedExecution(ThreadPoolExecutor.java:2055)
	at java.base/java.util.concurrent.ThreadPoolExecutor.reject(ThreadPoolExecutor.java:825)
	at java.base/java.util.concurrent.ThreadPoolExecutor.execute(ThreadPoolExecutor.java:1355)
	at org.apache.spark.executor.Executor.launchTask(Executor.scala:270)
	at org.apache.spark.scheduler.local.LocalEndpoint.$anonfun$reviveOffers$1(LocalSchedule

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/apache-spark/3.2.1/libexec/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [Errno 54] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/homebrew/Cellar/apache-spark/3.2.1/libexec/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/homebrew/Cellar/apache-spark/3.2.1/libexec/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 503, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending comman

ConnectionRefusedError: [Errno 61] Connection refused